# Movie-DB: A Centralized Movie Database
### Data Engineering Capstone Project

#### Project Summary
This database combines several ratings to create a centralized movie database. The database can be used for recommender systems, entertainment apps which require movie-data analytics or even for personal use when searching for a movie to watch. I use publicly available datasets on IMDb, RottenTomatoes, Oscars, Golden Globes from Kaggle and IMDb website to create the database. 

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

# Package Management for the Project
Since we load parquet file, I need to install fastparquet, which can be used by Pandas to read in the data. For that, we upgrade Numpy and then install fastparquet. We then import the necessary libraries needed

In [1]:
!pip install numpy --upgrade

    100% |████████████████████████████████| 13.4MB 1.2MB/s eta 0:00:01   16% |█████▏                          | 2.2MB 4.3MB/s eta 0:00:03    22% |███████▎                        | 3.1MB 18.8MB/s eta 0:00:01    43% |██████████████                  | 5.9MB 18.4MB/s eta 0:00:01    57% |██████████████████▎             | 7.7MB 19.7MB/s eta 0:00:01    88% |████████████████████████████▌   | 11.9MB 18.4MB/s eta 0:00:01
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
  Found existing installation: numpy 1.12.1
    Uninstalling numpy-1.12.1:
      Successfully uninstalled numpy-1.12.1


In [2]:
!pip install fastparquet

    100% |████████████████████████████████| 1.2MB 10.7MB/s ta 0:00:01    61% |███████████████████▉            | 727kB 15.5MB/s eta 0:00:01
    100% |████████████████████████████████| 122kB 17.6MB/s ta 0:00:01
    100% |████████████████████████████████| 9.5MB 3.2MB/s eta 0:00:01   4% |█▎                              | 389kB 19.2MB/s eta 0:00:01    13% |████▌                           | 1.3MB 19.1MB/s eta 0:00:01    43% |█████████████▉                  | 4.1MB 21.1MB/s eta 0:00:01    53% |█████████████████               | 5.1MB 17.5MB/s eta 0:00:01    62% |████████████████████            | 5.9MB 19.2MB/s eta 0:00:01    83% |██████████████████████████▋     | 7.9MB 18.2MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 10.8MB/s ta 0:00:01
    100% |████████████████████████████████| 1.5MB 10.3MB/s ta 0:00:01
    100% |████████████████████████████████| 256kB 10.3MB/s ta 0:00:01    74% |███████████████████████▉        | 184kB 20.3MB/s eta 0:00:01
  Running setup.py bdist_wheel 

In [3]:
# All the imports are done here
import pandas as pd
import os
import numpy as np
import re
import fastparquet
import psycopg2
from sql_queries import *

In [4]:
#This is needed when copying directly from numpy, since Postgres only supports native Python formats
import numpy as np
from psycopg2.extensions import register_adapter, AsIs

def addapt_numpy_float64(numpy_float64):
    return AsIs(numpy_float64)

def addapt_numpy_int64(numpy_int64):
    return AsIs(numpy_int64)

def addapt_numpy_float32(numpy_float32):
    return AsIs(numpy_float32)

def addapt_numpy_int32(numpy_int32):
    return AsIs(numpy_int32)

def addapt_numpy_array(numpy_array):
    return AsIs(tuple(numpy_array))

register_adapter(np.float64, addapt_numpy_float64)
register_adapter(np.int64, addapt_numpy_int64)
register_adapter(np.float32, addapt_numpy_float32)
register_adapter(np.int32, addapt_numpy_int32)
register_adapter(np.ndarray, addapt_numpy_array)

We will now create a database called moviedb, and create the necessary tables. This is all taken care of using some SQL queries which are defined in sql_queries.py. We have to run create_tables first, and only then proceed with the notebook.

In [5]:
%run create_tables.py

In [6]:
#Connecting to the database created in create_tables.py
conn = psycopg2.connect("host=127.0.0.1 dbname=moviedb user=student password=student")
conn.set_session(autocommit=True)
cur = conn.cursor()

In [7]:
def convert_nan(df):
    """
    This function converts nulls in a Pandas dataset to None object, which when saved in a CSV is blank. 
    This is done so that when the CSV is loaded, we correctly classify NULL using ""
    Input: A dataframe which contains some nulls
    Output: A dataframe in which all nulls are now None object
    """
    return df.where(pd.notnull(df), None)

# Step 1: Scope the Project and Gather Data

## Scope 
The database will contain information about movies, so TV Shows are not covered. The amount of movies in this database depends on the amount of movies in the datasets, and does not cover all the movies in the world. I plan to use public datasets that are available online, and try to pre-process them and combine them into a centralized database which can be used for analytics. Questions that can be answered include - <br/>
a) Which oscars was the film Titanic nominated for and which did it win? <br/>
b) What is the highest IMDb rated movie for a particular genre? <br/>
c) What is the highest rated movie that Steven Spielberg has directed? <br/>
<br/>
The possibilities are endless with more data. The end solution consists of a database in the Snowflake Schema with a fact table and multiple dimension tables which can be joined for more information. The tools I use include - Python and Postgres.

## Describe and Gather Data 
The datasets that I have gathered are all public datasets available on Kaggle and the IMDb website. The details are as follows - <br/> 
1) From https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset <br/> 
a) IMDb_movies.csv <br/>
b) IMDb_names.csv <br/>
2) From https://www.kaggle.com/stefanoleone992/rotten-tomatoes-movies-and-critic-reviews-dataset <br/>
a) rotten_tomatoes_movies.csv <br/>
3) From https://www.kaggle.com/unanimad/the-oscar-award <br/>
a) the_oscar_award.csv <br/>
4) From https://www.kaggle.com/unanimad/golden-globe-awards <br/>
a) golden_globe_awards.csv <br/>
5) From https://datasets.imdbws.com/ <br/>
a) cast_movies.parquet (I have downloaded the dataset and saved as parquet since its a huge dataset) <br/>
b) IMDB_Ratings.tsv 
<br/> <br/>
Thus, the requirement of having from atleast 2 different sources is also covered

In [8]:
# Read in the data here
movies_df = pd.read_csv('data/IMDb_movies.csv')
names_df = pd.read_csv('data/IMDb_names.csv')
ratings_df = pd.read_csv('data/IMDB_Ratings.tsv', sep="\t")
rt_df = pd.read_csv('data/rotten_tomatoes_movies.csv')
cast_df = pd.read_parquet("data/cast_movies.parquet")

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
oscar_df = pd.read_csv('data/the_oscar_award.csv')
gg_df = pd.read_csv('data/golden_globe_awards.csv')

# Step 2: Explore and Assess the Data
## Explore the Data 
We do some basic Exploratory Data Analysis, to check for missing values, duplicates and other potential data-quality issues

### Length of the Datasets
The datasets are pretty big, and easily cover the requirements - <br/>
a) The number of rows > 1 Million </br>
b) Atleast 2 different formats of data

In [10]:
print("Movies Dataset length : {}".format(len(movies_df)))
print("Names Dataset length : {}".format(len(names_df)))
print("Ratings Dataset length : {}".format(len(ratings_df)))
print("Rotten Tomatoes Dataset length : {}".format(len(rt_df)))
print("Cast Dataset length : {}".format(len(cast_df)))
print("Oscar Dataset length : {}".format(len(oscar_df)))
print("Golden Globes Dataset length : {}".format(len(gg_df)))

Movies Dataset length : 85855
Names Dataset length : 297705
Ratings Dataset length : 1172312
Rotten Tomatoes Dataset length : 17712
Cast Dataset length : 836122
Oscar Dataset length : 10395
Golden Globes Dataset length : 7991


### Checking for Missing Values
I check which columns contain missing values. This can help us determining which columns are useful and can be used in the datasets, also which can be used as Primary Keys, or where we can set constrains such as NOT NULL

The main dataset which will be used for the movies contains some nulls for fields like director, writer which will be resolved since I will join it with a much cleaner dataset. Most of the nulls come from income fields (which I will omit for my database), and metascore

In [11]:
for column in movies_df.columns:
    print(column, pd.isnull(movies_df[column]).sum())

imdb_title_id 0
title 0
original_title 0
year 0
date_published 0
genre 0
duration 0
country 64
language 833
director 87
writer 1572
production_company 4455
actors 69
description 2115
avg_vote 0
votes 0
budget 62145
usa_gross_income 70529
worlwide_gross_income 54839
metascore 72550
reviews_from_users 7597
reviews_from_critics 11797


A lot of people are still alive so all columns related to death will be NULL. A lot of them are missing height or birth details, possibly because they are not easily available

In [12]:
for column in names_df.columns:
    print(column, pd.isnull(names_df[column]).sum())

imdb_name_id 0
name 0
birth_name 0
height 253024
bio 93007
birth_details 187093
date_of_birth 187093
place_of_birth 193713
death_details 257772
date_of_death 257772
place_of_death 260667
reason_of_death 275011
spouses_string 252353
spouses 0
divorces 0
spouses_with_children 0
children 0


The next 2 datasets are from IMDb website itself, which means they are very clean, very high quality datasets, as we can see no missing values

In [13]:
for column in ratings_df.columns:
    print(column, pd.isnull(ratings_df[column]).sum())

tconst 0
averageRating 0
numVotes 0


In [14]:
for column in cast_df.columns:
    print(column, pd.isnull(cast_df[column]).sum())

imdb_title_id 0
tconst 0
ordering 0
nconst 0
category 0
job 0
characters 0


Rotten Tomatoes dataset has less movies than the IMDb dataset, which means not every movie in IMDb dataset will get a RottenTomatoes rating. Except for critics_consensus, most of the dataset looks relatively clean. A lot of repeating fields here won't be considered since I will already take those from the IMDb dataset.

In [15]:
for column in rt_df.columns:
    print(column, pd.isnull(rt_df[column]).sum())

rotten_tomatoes_link 0
movie_title 0
movie_info 321
critics_consensus 8578
content_rating 0
genres 19
directors 194
authors 1542
actors 352
original_release_date 1166
streaming_release_date 384
runtime 314
production_company 499
tomatometer_status 44
tomatometer_rating 44
tomatometer_count 44
audience_status 448
audience_rating 296
audience_count 297
tomatometer_top_critics_count 0
tomatometer_fresh_critics_count 0
tomatometer_rotten_critics_count 0


The Awards Datasets are very clean and also very small, since they only contain information about the awardees and nominees

In [16]:
for column in oscar_df.columns:
    print(column, pd.isnull(oscar_df[column]).sum())

year_film 0
year_ceremony 0
ceremony 0
category 0
name 0
film 304
winner 0


In [34]:
for column in gg_df.columns:
    print(column, pd.isnull(gg_df[column]).sum())

year_film 0
year_award 0
ceremony 0
category 0
nominee 0
film 1800
win 0


### Checking for Duplicated Data
We see that Oscars dataset contains some duplicates which we will remove in the cleaning set. Other duplicates might be possible, but those will be taken care in the data pipelines

In [35]:
print("Movies Duplicates : {}".format(len(movies_df[movies_df.duplicated(keep=False)==True])/2))
print("Names Duplicates : {}".format(len(names_df[names_df.duplicated(keep=False)==True])/2))
print("Ratings Duplicates : {}".format(len(ratings_df[ratings_df.duplicated(keep=False)==True])/2))
print("Rotten Tomatoes Duplicates : {}".format(len(rt_df[rt_df.duplicated(keep=False)==True])/2))
print("Oscars Duplicates : {}".format(len(oscar_df[oscar_df.duplicated(keep=False)==True])/2))
print("Golden Globe Duplicates : {}".format(len(gg_df[gg_df.duplicated(keep=False)==True])/2))
print("Cast Duplicates : {}".format(len(cast_df[cast_df.duplicated(keep=False)==True])/2))

Movies Duplicates : 0.0
Names Duplicates : 0.0
Ratings Duplicates : 0.0
Rotten Tomatoes Duplicates : 0.0
Oscars Duplicates : 5.5
Golden Globe Duplicates : 0.0
Cast Duplicates : 0.0


### Checking for Data Quality Issues 
Since some of the datasets sourced from Kaggle have been scraped from the internet, they will have data quality issues. We need to be aware of the relevant ones so that we can resolve those before our data pipelines

#### Issue with Cases 
Since the movies in different datasets will be in different cases, we will have to convert all the movies to lower case during processing

#### Other Issues
Some of the issues came while working with the project which I will mention here and how I tackle them

In [19]:
#We see that some of the rows contain a value "TV Movie 2019" for year, which will need to be cleaned
x1 = movies_df.copy()
x1["year"] = x1.year.astype(str)
np.unique(x1.year)

array(['1894', '1906', '1911', '1912', '1913', '1914', '1915', '1916',
       '1917', '1918', '1919', '1920', '1921', '1922', '1923', '1924',
       '1925', '1926', '1927', '1928', '1929', '1930', '1931', '1932',
       '1933', '1934', '1935', '1936', '1937', '1938', '1939', '1940',
       '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948',
       '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956',
       '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964',
       '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972',
       '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980',
       '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
       '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
      

In [36]:
#We see some issues in the Golden Globes Dataset which will be taken care of in the pipeline - 
#Whenever there is a comma and after the comma its either 'The' or 'A', the film name is inverted so we need to correct that
list(np.unique(gg_df.film.dropna()))[:5]

[' Remains of the Day, The',
 ' Roma ',
 '$1,000,000 Duck',
 "'Round Midnight",
 "'night, Mother"]

## Cleaning Steps
Document steps necessary to clean the data

In [21]:
#As we saw, we need to clean the year column of movies_df 
movies_df.loc[movies_df.year == "TV Movie 2019", "year"] = 2019

In [22]:
#To clean the Golden Globes dataset, I use some custom defined functions
def process_comma(x):
    """
    This function is necessary since we need to change the order of the words in the dataset when there is a comma
    The condition is that there should be only 2 substrings (one before and one after the comma), and if there is 
    'a' or 'the' in the end of the string. 
    Input: A string which needs to be processed 
    Output: A string which is corrected for the issue due to the comma
    """
    if "," in x:
        x1 = x.split(",")
        if ((x1[-1] == " a") or (x1[-1] == " the")) and (len(x1)==2):
            return x1[1][1:] + " " + x1[0]
    return x

In [23]:
oscar_df = oscar_df.drop_duplicates()

There needs to be more cleaning just as changing column names, and dropping duplicates, but all that will be performed in the data pipelines

# Step 3: Define the Data Model
## 3.1 Conceptual Data Model

Map out the conceptual data model and explain why you chose that model

I will use a snowflake schema with a fact table and multiple dimension tables. The diagram is as follows - 
![title](schema.PNG)

As we can see, the Movies table will be the fact table that contains information about the movie, its run-time, release date, IMDB score, Rotten Tomatoes scores, number of oscars won and number of golden globes won. The Crew table will contain a mapping from movies to names (names of actors, directors etc.) which are stored in the Names table. The Characters table contains all the characters that are played by the actors in the movies. The Oscars and Golden Globes table contain the winners and nominees for the movies, while Genres table is a mapping between movies and genres. The votes table stores the votes that each movie gets.  <br/> 

I have chosen this model since it is 3NF normalized and it allows me to get answers quickly based on various joins. The information is stored in such a way that each table can be updated independently. It also makes it easier to visualize and create new queries.

## 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

To transfor our current dataset into the following database diagram that I have developed, we will follow the following steps - <br/> <br/>
![title](etl_pipeline.png)

<br/>
After loading all the files and doing some basic pre-processing, we will follow certain steps to transform each file into the desired tables - <br/>
1) Movies table will be formed by joining movies_df with ratings_df, rt_df, oscar_df and gg_df. <br/>
2) Names table is readily available as names_df <br/>
3) Crew table is also available as cast_df <br/>
4) For Characters table, we have to extract the characters from the cast_df for each title <br/>
5) Genre table will also be created from movies_df <br/>
6) Oscars and Golden_Globes tables are readily available as gg_df and oscar_df after some processing <br/>
7) Votes table will also be extracted from movies_df and rt_df <br/>

# Step 4: Run Pipelines to Model the Data 
## 4.1 Create the data model
Build the data pipelines to create the data model.

### Processing Movies Data

In [25]:
dt1 = movies_df[["imdb_title_id", "original_title", "year", "duration", "country", "description", "metascore"]].copy()
#Converting to lower case to avoid any joining issues with names
dt1["original_title"] = dt1.original_title.str.lower()
dt1 = dt1.rename(columns={"original_title":'Title'})

In [26]:
#Joining the ratings table to get the correct IMDb score
ratings_df = ratings_df.rename(columns={"tconst":'imdb_title_id', "averageRating":'IMDB_rating'})
dt2 = dt1.merge(ratings_df, on="imdb_title_id")

In [27]:
#Processing the Rotten Tomatoes file, so that it can be merged with the movies file
rt_df2 = rt_df[["movie_title", "original_release_date", "content_rating", "tomatometer_status", "tomatometer_rating", "tomatometer_count", "audience_status", "audience_rating",  "audience_count"]].copy()
#Following the same principle of converting tiles to lower case
rt_df2["movie_title"] = rt_df2["movie_title"].str.lower()
rt_df2 = rt_df2.rename(columns={"movie_title":'Title'})
rt_df2["date_dt"] = pd.to_datetime(rt_df2["original_release_date"]) 
rt_df2["year"] = rt_df2.date_dt.apply(lambda x: x.year)
rt_df2 = rt_df2.drop(columns=["original_release_date", "date_dt"])

In [28]:
dt2["year"] = dt2.year.astype(float)
dt3 = dt2.merge(rt_df2, on=["Title", "year"], how="left")

### Creating Oscars Table
We will just process oscar_df a bit and join it with movies_df to get the imdb_title_id which is our main key

In [29]:
oscar_df = oscar_df.dropna(subset=['film'])
oscar_df = oscar_df.rename(columns={"film":"Title", "year_film":'year'})
oscar_df["Title"] = oscar_df["Title"].str.lower()
o_df1 = oscar_df.merge(dt3[["imdb_title_id", "Title", "year"]], on=["Title", "year"], how="left")

In [30]:
o_df1.head()

,year,year_ceremony,ceremony,category,name,Title,winner,imdb_title_id
0,1927,1928,1,ACTOR,Richard Barthelmess,the noose,False,NaN
1,1927,1928,1,ACTOR,Emil Jannings,the last command,True,NaN
2,1927,1928,1,ACTRESS,Louise Dresser,a ship comes in,False,NaN
3,1927,1928,1,ACTRESS,Janet Gaynor,7th heaven,True,tt0018379
4,1927,1928,1,ACTRESS,Gloria Swanson,sadie thompson,False,NaN


In [31]:
for column in o_df1.columns:
    print(column, pd.isnull(o_df1[column]).sum())

year 0
year_ceremony 0
ceremony 0
category 0
name 0
Title 0
winner 0
imdb_title_id 2900


In [32]:
#When inserting NaN's, we have to first convert it to None so SQL translates it as NULL
#We save the processed table in output_files
o_df1 = convert_nan(o_df1)
o_df1.to_csv('output_files/Oscars.csv', header=False, index=False, sep="|")

### Creating Golden_Globes Table
Same as the oscars table, we have to process the Golden Globes table a bit, but this time, its a bit more code since the file is more unstructured

In [56]:
gg_only_movies = gg_df[~gg_df.category.str.contains("Television")].copy()
gg_only_movies["film"] = np.where(pd.isnull(gg_only_movies.film), gg_only_movies.nominee, gg_only_movies.film)

In [57]:
gg_only_movies = gg_only_movies.rename(columns={"film":"Title", "year_film":'year'})
gg_only_movies["Title"] = gg_only_movies["Title"].str.lower().str.strip()
gg_only_movies["Title"] = gg_only_movies["Title"].apply(lambda x: process_comma(x))

In [58]:
gg_df1 = gg_only_movies.merge(dt3[["imdb_title_id", "Title", "year"]], on=["Title", "year"], how="left")

In [59]:
gg_df1.head()

,year,year_award,ceremony,category,nominee,Title,win,imdb_title_id
0,1943,1944,1,Best Performance by an Actress in a Supporting...,Katina Paxinou,for whom the bell tolls,True,tt0035896
1,1943,1944,1,Best Performance by an Actor in a Supporting R...,Akim Tamiroff,for whom the bell tolls,True,tt0035896
2,1943,1944,1,Best Director - Motion Picture,Henry King,the song of bernadette,True,tt0036377
3,1943,1944,1,Picture,The Song Of Bernadette,the song of bernadette,True,tt0036377
4,1943,1944,1,Actress In A Leading Role,Jennifer Jones,the song of bernadette,True,tt0036377


In [61]:
for column in gg_df1.columns:
    print(column, pd.isnull(gg_df1[column]).sum())

year 0
year_award 0
ceremony 0
category 0
nominee 0
Title 0
win 0
imdb_title_id 948


In [62]:
#When inserting NaN's, we have to first convert it to None so SQL translates it as NULL
gg_df1 = convert_nan(gg_df1)
gg_df1 = gg_df1.drop_duplicates(subset=["year","year_award","ceremony","category","nominee","Title","win"])
str_col_list = ["category","nominee","Title"]
for column in str_col_list:
    gg_df1[column] = gg_df1[column].str.strip()
gg_df1.to_csv('output_files/Golden_Globes.csv', header=False, index=False, sep="|")

### Creating Movies Table 
Now that we have processed the Oscars and Golden Globes table, we can aggregate it to calculate additional fields, and add that to our fact table

In [63]:
o_df2 = o_df1.groupby(by=["imdb_title_id", "year"], as_index=False).agg({"winner":'sum', "Title":'count'})
o_df2 = o_df2.rename(columns={"winner":"num_oscars", "Title":'num_nominations_oscars'})

In [64]:
gg_df2 = gg_df1.groupby(by=["imdb_title_id", "year"], as_index=False).agg({"win":'sum', "Title":'count'})
gg_df2 = gg_df2.rename(columns={"win":"num_golden_globes", "Title":'num_nominations_globes'})

In [65]:
dt4 = dt3.merge(o_df2, on=["imdb_title_id", "year"], how="left").merge(gg_df2, on=["imdb_title_id", "year"], how="left")
dt4[["num_oscars", "num_nominations_oscars", "num_golden_globes", "num_nominations_globes"]] = dt4[["num_oscars", "num_nominations_oscars", "num_golden_globes", "num_nominations_globes"]].fillna(0)

In [66]:
movies = dt4[['imdb_title_id', 'Title', 'year', 'duration', 'country', 'description',
              'metascore', 'IMDB_rating', 'content_rating', 'tomatometer_status', 'tomatometer_rating', 
              'audience_status', 'audience_rating', 'num_oscars',
              'num_nominations_oscars', 'num_golden_globes', 'num_nominations_globes']].copy()

In [67]:
movies.head()

,imdb_title_id,Title,year,duration,country,description,metascore,IMDB_rating,content_rating,tomatometer_status,tomatometer_rating,audience_status,audience_rating,num_oscars,num_nominations_oscars,num_golden_globes,num_nominations_globes
0,tt0000009,miss jerry,1894.0,45,USA,The adventures of a female reporter in the 1890s.,NaN,5.8,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
1,tt0000574,the story of the kelly gang,1906.0,70,Australia,True story of notorious Australian outlaw Ned ...,NaN,6.1,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
2,tt0001892,den sorte drøm,1911.0,53,"Germany, Denmark",Two men of high rank are both wooing the beaut...,NaN,5.9,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
3,tt0002101,cleopatra,1912.0,100,USA,The fabled queen of Egypt's affair with Roman ...,NaN,5.1,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
4,tt0002130,l'inferno,1911.0,68,Italy,Loosely adapted from Dante's Divine Comedy and...,NaN,7.1,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0


In [68]:
for column in movies.columns:
    print(column, pd.isnull(movies[column]).sum())

imdb_title_id 0
Title 0
year 0
duration 0
country 64
description 2115
metascore 72539
IMDB_rating 0
content_rating 76450
tomatometer_status 76459
tomatometer_rating 76459
audience_status 76577
audience_rating 76473
num_oscars 0
num_nominations_oscars 0
num_golden_globes 0
num_nominations_globes 0


In [69]:
#When inserting NaN's, we have to first convert it to None so SQL translates it as NULL
movies = convert_nan(movies)
movies = movies.drop_duplicates(subset=["imdb_title_id"])
movies.to_csv('output_files/Movies.csv', header=False, index=False, sep="|")

### Creating Other Dimension Tables

### Creating Votes Table
We will extract the votes information from dt4 so that we can update this table without affecting the fact table too much

In [70]:
votes = dt4[['imdb_title_id', 'numVotes', 'tomatometer_count', 'audience_count']].copy()
votes = votes.rename(columns={"numVotes":'IMDB_votes', "tomatometer_count":'RT_votes_critics', "audience_count":'RT_votes_audience'})

In [71]:
votes.head()

,imdb_title_id,IMDB_votes,RT_votes_critics,RT_votes_audience
0,tt0000009,156,NaN,NaN
1,tt0000574,643,NaN,NaN
2,tt0001892,195,NaN,NaN
3,tt0002101,463,NaN,NaN
4,tt0002130,2500,NaN,NaN


In [72]:
for column in votes.columns:
    print(column, pd.isnull(votes[column]).sum())

imdb_title_id 0
IMDB_votes 0
RT_votes_critics 76459
RT_votes_audience 76474


In [73]:
votes = convert_nan(votes)
votes = votes.drop_duplicates(subset=["imdb_title_id"])
votes.to_csv('output_files/Votes.csv', header=False, index=False, sep="|")

## Creating Genres Table
Since we want our database to be normalized, there needs to be a separate Genre table which lists for each movie, all the different genres. We will extract this information from movie_df

In [74]:
genre_df1 = movies_df[["imdb_title_id", "genre"]].copy()
list_movies = []
list_genres = []
for i in range(len(genre_df1)):
    for genre in genre_df1["genre"][i].split(","):
        list_movies.append(genre_df1["imdb_title_id"][i])
        list_genres.append(genre.strip())

In [75]:
assert(len(list_movies)==len(list_genres))
print(np.unique(list_genres))

['Action' 'Adult' 'Adventure' 'Animation' 'Biography' 'Comedy' 'Crime'
 'Documentary' 'Drama' 'Family' 'Fantasy' 'Film-Noir' 'History' 'Horror'
 'Music' 'Musical' 'Mystery' 'News' 'Reality-TV' 'Romance' 'Sci-Fi'
 'Sport' 'Thriller' 'War' 'Western']


In [76]:
genre_df2 = pd.DataFrame()
genre_df2["Genre"] = pd.Series(list_genres)
genre_df2["imdb_title_id"] = pd.Series(list_movies)

In [77]:
genre_df2.head()

,Genre,imdb_title_id
0,Romance,tt0000009
1,Biography,tt0000574
2,Crime,tt0000574
3,Drama,tt0000574
4,Drama,tt0001892


In [78]:
genre_df2 = genre_df2.drop_duplicates()
genre_df2.to_csv('output_files/Genres.csv', header=False, index=False, sep="|")

## Creating Crew Table
The cast_movies.parquet file already contains the format we want, but we just have to pre-process the data a bit to create a list for characters, which would later be used for the characters dataset

In [79]:
def get_list_of_characters(x):
    y = []
    if(type(x) != list):
        print(x)
    for i in x:
        y.append(i.replace("[","").replace("]","").strip('"').strip(" ").replace("\\N", ""))
    return y

In [80]:
cast_df1 = cast_df.drop(columns=["tconst"])
cast_df1["characters"] = cast_df1.characters.str.split(",")
cast_df1["characters"] = cast_df1.characters.apply(lambda x: get_list_of_characters(x))

In [81]:
cast_df2 = cast_df1.replace("\\N", np.NaN).rename(columns={"nconst":"imdb_name_id"})
cast_df2 = cast_df2.drop_duplicates(subset=["imdb_title_id", "imdb_name_id", "category"])

In [82]:
cast_df2.head()

,imdb_title_id,ordering,imdb_name_id,category,job,characters
0,tt0000009,1,nm0063086,actress,NaN,[Miss Geraldine Holbrook (Miss Jerry)]
1,tt0000009,2,nm0183823,actor,NaN,[Mr. Hamilton]
2,tt0000009,3,nm1309758,actor,NaN,[Chauncey Depew - the Director of the New York...
3,tt0000009,4,nm0085156,director,NaN,[]
4,tt0000574,10,nm0675239,cinematographer,director of photography,[]


In [83]:
for column in cast_df2.columns:
    print(column, pd.isnull(cast_df2[column]).sum())

imdb_title_id 0
ordering 0
imdb_name_id 0
category 0
job 621377
characters 0


In [84]:
crew = cast_df2.drop(columns=["characters"])
crew = crew.drop_duplicates(subset=["imdb_title_id", "imdb_name_id", "category"])
crew.to_csv('output_files/Crew.csv', header=False, index=False, sep="|")

## Creating Names Table
For actual information on the cast, the IMDb_names.csv file contains all the information we need

In [85]:
names_df1 = names_df.drop(columns = ['spouses_string', 'spouses', 'divorces', 'spouses_with_children', 'children'])

In [86]:
names_df1.head()

,imdb_name_id,name,birth_name,height,bio,birth_details,date_of_birth,place_of_birth,death_details,date_of_death,place_of_death,reason_of_death
0,nm0000001,Fred Astaire,Frederic Austerlitz Jr.,177.0,"Fred Astaire was born in Omaha, Nebraska, to J...","May 10, 1899 in Omaha, Nebraska, USA",1899-05-10,"Omaha, Nebraska, USA","June 22, 1987 in Los Angeles, California, USA ...",1987-06-22,"Los Angeles, California, USA",pneumonia
1,nm0000002,Lauren Bacall,Betty Joan Perske,174.0,Lauren Bacall was born Betty Joan Perske on Se...,"September 16, 1924 in The Bronx, New York City...",1924-09-16,"The Bronx, New York City, New York, USA","August 12, 2014 in New York City, New York, US...",2014-08-12,"New York City, New York, USA",stroke
2,nm0000003,Brigitte Bardot,Brigitte Bardot,166.0,"Brigitte Bardot was born on September 28, 1934...","September 28, 1934 in Paris, France",1934-09-28,"Paris, France",NaN,NaN,NaN,NaN
3,nm0000004,John Belushi,John Adam Belushi,170.0,"John Belushi was born in Chicago, Illinois, US...","January 24, 1949 in Chicago, Illinois, USA",1949-01-24,"Chicago, Illinois, USA","March 5, 1982 in Hollywood, Los Angeles, Calif...",1982-03-05,"Hollywood, Los Angeles, California, USA",acute cocaine and heroin intoxication
4,nm0000005,Ingmar Bergman,Ernst Ingmar Bergman,179.0,"Ernst Ingmar Bergman was born July 14, 1918, t...","July 14, 1918 in Uppsala, Uppsala län, Sweden",1918-07-14,"Uppsala, Uppsala län, Sweden","July 30, 2007 in Fårö, Gotlands län, Sweden (...",2007-07-30,"Fårö, Gotlands län, Sweden",natural causes


In [87]:
for column in names_df1.columns:
    print(column, pd.isnull(names_df1[column]).sum())

imdb_name_id 0
name 0
birth_name 0
height 253024
bio 93007
birth_details 187093
date_of_birth 187093
place_of_birth 193713
death_details 257772
date_of_death 257772
place_of_death 260667
reason_of_death 275011


In [88]:
def remove_alphabets(x):
    """
    We have some alphabets in 2 columns - date_of_birth / date_of_death so we use regex to just remove those 
    Input: Unprocessed string 
    Output: Processed string without alphabets
    """
    if not pd.isnull(x):
        return re.sub(r'[A-Za-z]', '', x).strip()
    else:
        return x

In [89]:
names_df1["date_of_birth"] = names_df1["date_of_birth"].apply(lambda x: remove_alphabets(x))
names_df1["date_of_death"] = names_df1["date_of_death"].apply(lambda x: remove_alphabets(x))

In [90]:
names_df1 = names_df1.drop_duplicates(subset=["imdb_name_id"])
names_df1.to_csv('output_files/Names.csv', header=False, index=False, sep="|")

## Creating Characters Table
We map for a particular movie, for a particular person and for a particular category, what characters the person plays. We do this so that the database is still normalized.

In [91]:
#Each character gets one row
character_df = cast_df2[cast_df2.characters.apply(lambda x: x!=[""])].copy().reset_index(drop=True)
character_df = character_df[["imdb_title_id", "imdb_name_id", "category", "characters"]].copy()
list_movies = []
list_names = []
list_category = []
list_characters = []
for i in range(len(character_df)):
    for character in character_df["characters"][i]:
        list_movies.append(character_df["imdb_title_id"][i])
        list_names.append(character_df["imdb_name_id"][i])
        list_category.append(character_df["category"][i])
        list_characters.append(character)

In [92]:
assert(len(list_movies) == len(list_names))
assert(len(list_names) == len(list_category))
assert(len(list_category)==len(list_characters))
character_df2 = pd.DataFrame()
character_df2["imdb_title_id"] = list_movies
character_df2["imdb_name_id"] = list_names
character_df2["category"] = list_category
character_df2["character"] = list_characters

In [93]:
character_df2.head()

,imdb_title_id,imdb_name_id,category,character
0,tt0000009,nm0063086,actress,Miss Geraldine Holbrook (Miss Jerry)
1,tt0000009,nm0183823,actor,Mr. Hamilton
2,tt0000009,nm1309758,actor,Chauncey Depew - the Director of the New York ...
3,tt0000574,nm0846887,actress,Kate Kelly
4,tt0000574,nm0846894,actor,School Master


In [94]:
for column in character_df2.columns:
    print(column, pd.isnull(character_df2[column]).sum())

imdb_title_id 0
imdb_name_id 0
category 0
character 0


In [95]:
character_df2.to_csv('output_files/Characters.csv', header=False, index=False, sep="|")

### Loading our processed data to the tables

In [97]:
def load_table(table_name, file_path, cur):
    f = open(file_path, 'r')
    cur.copy_from(f, table_name, sep="|", null='')
    f.close()

In [98]:
tables = ["Movies", "Crew", "Names", "Characters", "Genres", "Votes", "Oscars", "Golden_Globes"]
for table_name in tables:
    print("Inserting data into {}".format(table_name))
    file_path = 'output_files/{}.csv'.format(table_name)
    load_table(table_name, file_path, cur)

Inserting data into Movies
Inserting data into Crew
Inserting data into Names
Inserting data into Characters
Inserting data into Genres
Inserting data into Votes
Inserting data into Oscars
Inserting data into Golden_Globes


## 4.2 Data Quality Checks
Data Quality Checks are important since they will tell us whether our ETL pipeline worked as expected. I will perform 2 checks on data quality and 1 check on looking at the data for a particular movie - Inception


### Quality Check for Nulls for Genres Table
We saw that Genres table did not have any nulls, so lets check that if it is true in the database

In [102]:
#We expect no result from this query and indeed, this is what we see
cur.execute("SELECT * FROM Genres where genre IS NULL")
cur.fetchall()

[]

### Quality Check for Total Rows for Oscars Table
As expected, we see the same number of rows in the database and our dataframe

In [106]:
cur.execute("SELECT COUNT(*) FROM Oscars")
cur.fetchall()

[(10085,)]

In [107]:
print(len(o_df1))

10085


### Fact Table Check for Movie Inception
I want to just see the data that our database contains for one of my favorite movies!

In [108]:
cur.execute(""" 
SELECT * from Movies where title = 'inception'
""")
cur.fetchall()

[('tt1375666',
  'inception',
  2010,
  148,
  'USA, UK',
  'A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O.',
  74.0,
  8.8,
  'PG-13',
  'Certified-Fresh',
  87.0,
  'Upright',
  91.0,
  4,
  8,
  0,
  4)]

Thus we see the rating, Rotten Tomato rating, the number of Oscars won (4). Let's also see what Genre it is 

In [110]:
cur.execute(""" 
SELECT genre, title from Movies M
JOIN Genres G on G.imdb_title_id = M.imdb_title_id
where title = 'inception'
""")
cur.fetchall()

[('Action', 'inception'), ('Adventure', 'inception'), ('Sci-Fi', 'inception')]

So we see that Inception is classified as Action, Adventure & Sci-Fi

## 4.3 Data dictionary 
The Data Dictionary is in the Excel File - Data Dictionary.xlsx. Each tab contains the dictionary for one table

# Step 5: Complete Project Write Up
<b>- Clearly state the rationale for the choice of tools and technologies for the project. </b> <br/>
I chose to code the project in Python because of the ease of using the language and the variety of tools. Since I wanted to create a Relational Database, I chose to use Postgres because of its easy support with Python <br/> <br/>

<b>* Propose how often the data should be updated and why.</b> <br/>
The tables such as Movies, Names, Votes etc should ideally be updated daily, since there can always be new movies and new ratings. For tables such as Oscars and Golden Globes, they can be updated once every year. <br/> <br/>

<b>* Write a description of how you would approach the problem differently under the following scenarios:</b> <br/>
 <b>* The data was increased by 100x.</b>
 If the data is increased by 100x which is possible if you include all the movies in the world, the problem would be approached a bit differently. I would use Spark for all the processing, and store parquet files. For reliability, elasticity and security, moving the solution to a Cloud Provider like AWS would have to be considered <br/>
 
 <b>* The data populates a dashboard that must be updated on a daily basis by 7am every day.</b>
 If there is a dashboard that must be updated everyday, there would need to be an Airflow pipeline that would update the database every night, so that there is always the latest information. <br/>
 
 <b>* The database needed to be accessed by 100+ people.</b>
 If we need to manage the access rights of the database and the people, then its better to use Cloud services. 